In [1]:
from functools import partial
from typing import Literal
from dataclasses import dataclass

import pandas as pd
import redis

In [2]:
r = redis.Redis(db=4)

keys = r.keys()

In [3]:
keys[0].decode("utf-8")

'تا نشانده\u200cام در دل ساق سرو و سیمینتچیده\u200cام به هر دستی میوه\u200cهای سیمین را'

In [4]:
r.hgetall(keys[0])

{b'url': b'https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh32/',
 b'poet': b'\xd9\x81\xd8\xb1\xd9\x88\xd8\xba\xdb\x8c \xd8\xa8\xd8\xb3\xd8\xb7\xd8\xa7\xd9\x85\xdb\x8c',
 b'm1': b'\xd8\xaa\xd8\xa7 \xd9\x86\xd8\xb4\xd8\xa7\xd9\x86\xd8\xaf\xd9\x87\xe2\x80\x8c\xd8\xa7\xd9\x85 \xd8\xaf\xd8\xb1 \xd8\xaf\xd9\x84 \xd8\xb3\xd8\xa7\xd9\x82 \xd8\xb3\xd8\xb1\xd9\x88 \xd9\x88 \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd9\x86\xd8\xaa',
 b'm2': b'\xda\x86\xdb\x8c\xd8\xaf\xd9\x87\xe2\x80\x8c\xd8\xa7\xd9\x85 \xd8\xa8\xd9\x87 \xd9\x87\xd8\xb1 \xd8\xaf\xd8\xb3\xd8\xaa\xdb\x8c \xd9\x85\xdb\x8c\xd9\x88\xd9\x87\xe2\x80\x8c\xd9\x87\xd8\xa7\xdb\x8c \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd9\x86 \xd8\xb1\xd8\xa7'}

### Here we write some structures and functions for handling Beyt objects

In [5]:
def load_beyt(data: dict[bytes, bytes]) -> dict[str, str]:
    return {k.decode("utf-8"): v.decode("utf-8") for k, v in data.items()}


@dataclass
class Beyt:
    m1: str
    m2: str
    poet: str
    url: str

    @property
    def key(self):
        return self.m1 + self.m2

In [6]:
Beyt(**load_beyt(r.hgetall(keys[0])))

Beyt(m1='تا نشانده\u200cام در دل ساق سرو و سیمینت', m2='چیده\u200cام به هر دستی میوه\u200cهای سیمین را', poet='فروغی بسطامی', url='https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh32/')

In [7]:
db = []

for key in keys:
    db.append(Beyt(**load_beyt(r.hgetall(key))))

## Start Create DataFrame

In [71]:
df = pd.DataFrame(data=db)
df

,m1,m2,poet,url
0,تا نشانده‌ام در دل ساق سرو و سیمینت,چیده‌ام به هر دستی میوه‌های سیمین را,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...
1,الا ای رو ترش کرده که تا نبود مرا مدخل,نبشته گرد روی خود صلا نعم الادام الخل,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
2,تر کنم از آب چشم روی چونان خشک را,چون جگری بیش نیست سوخته بر خوان دل,سیف فرغانی,https://ganjoor.net/seyf/divan-seyf/ghazalha/s...
3,ای دل غم زده از عجز تو معلومم شد,که بر تیغ محبت سپر انداخته‌ای,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...
4,از می و مطرب مکن، مدعیا منع من,تا غزلی‌تر بود، قول تو خواهم شنید,سلمان ساوجی,https://ganjoor.net/salman/divanss/ghazalss/sh...
...,...,...,...,...
74092,صد نامه فرستادم صد راه نشان دادم,یا راه نمی‌دانی یا نامه نمی‌خوانی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...
74093,آن که دل من چو گوی در خم چوگان اوست,موقف آزادگان بر سر میدان اوست,سعدی,https://ganjoor.net/saadi/divan/ghazals/sh95/
74094,خون در دل و ریشهٔ تنم سوخت چنان,کز دیده بجای اشک خاکستر ریخت,ابوسعید ابوالخیر,https://ganjoor.net/abusaeed/robaee-aa/sh44/
74095,گردون برای خیمه خورشید فلکه‌ات,از کوه و ابر ساخته نازیر و سایه‌بان,حافظ,https://ganjoor.net/hafez/ghaside/sh1/


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74097 entries, 0 to 74096
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   m1      74097 non-null  object
 1   m2      74097 non-null  object
 2   poet    74097 non-null  object
 3   url     74097 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [73]:
# Find Counts of Poems URL

df.url.value_counts()

## IF YOU COLLECTED ENORMOUS DATA OF A POEM URL THEN YOU CAUGHT ALL BEYTS(COUPLET)  :)))

url
https://ganjoor.net/hafez/saghiname/                      58
https://ganjoor.net/moulavi/shams/ghazalsh/sh1940/        46
https://ganjoor.net/hafez/ghaside/sh3/                    43
https://ganjoor.net/hafez/ghaside/sh2/                    40
https://ganjoor.net/hafez/ghaside/sh1/                    40
                                                          ..
https://ganjoor.net/saeb/divan-saeb/takbeit/sh375/         1
https://ganjoor.net/moulavi/shams/robaeesh/sh466/          1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh1085/        1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh1502/        1
https://ganjoor.net/khosro/gozide/ghazal-khosro/sh317/     1
Name: count, Length: 14891, dtype: int64

In [74]:
# Get All Hafez Poems from db

df[df.poet == "حافظ"] # == df[df.poet.str.contains("حافظ")]

,m1,m2,poet,url
78,در چین زلفش ای دل مسکین چگونه‌ای,کشفته گفت باد صبا شرح حال تو,حافظ,https://ganjoor.net/hafez/ghazal/sh408/
84,سحرگهم چه خوش آمد که بلبلی گلبانگ,به غنچه می‌زد و می‌گفت در سخنرانی,حافظ,https://ganjoor.net/hafez/ghaside/sh2/
85,شود چون بید لرزان سرو آزاد,اگر بیند قد دلجوی فرخ,حافظ,https://ganjoor.net/hafez/ghazal/sh99/
95,بر طرف گلشنم گذر افتاد وقت صبح,آن دم که کار مرغ سحر آه و ناله بود,حافظ,https://ganjoor.net/hafez/ghazal/sh214/
137,مستی به آب یک دو عنب وضع بنده نیست,من سالخورده پیر خرابات پرورم,حافظ,https://ganjoor.net/hafez/ghazal/sh329/
...,...,...,...,...
74025,سوختم در چاه صبر از بهر آن شمع چگل,شاه ترکان فارغ است از حال ما کو رستمی,حافظ,https://ganjoor.net/hafez/ghazal/sh470/
74026,بخت حافظ گر از این گونه مدد خواهد کرد,زلف معشوقه به دست دگران خواهد بود,حافظ,https://ganjoor.net/hafez/ghazal/sh205/
74070,هم عفاالله صبا کز تو پیامی می‌داد,ور نه در کس نرسیدیم که از کوی تو بود,حافظ,https://ganjoor.net/hafez/ghazal/sh210/
74081,به خلدم دعوت ای زاهد مفرما,که این سیب زنخ زان بوستان به,حافظ,https://ganjoor.net/hafez/ghazal/sh419/


#### cleaning m1, m1

In [75]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,m1,m2,poet,url
1652,...,زنده مانم چو در آمدز در معشوقی,محتشم کاشانی,https://ganjoor.net/mohtasham/divan-moh/ghazal-moh/sh574/
62502,***,ای فخر من سلطان من سلطان من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1785/


In [76]:
df.loc[62502, "m1"] = "ای فخر من سلطان من سلطان من"
df.loc[62502, "m2"] = "فرمان ده و خاقان من خاقان من"

df.drop(1652, inplace=True)

df.reset_index(inplace=True)
##  Full Handy & IRREVERSIBLE!
# Remove this part!

In [77]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,index,m1,m2,poet,url


In [78]:
ocr_err = df[df.m1 == "ای دل دو سه شام تا سحرگاه مخسب"]
print(ocr_err[["m1", "m2"]])
if not ocr_err.empty:
    idx = ocr_err.index
    df.loc[idx, "m2"] = "در فرقت آفتاب چون ماه مخسب"
    print("FIXED")
    print(df.loc[idx, ["m1", "m2"]])


                                   m1                          m2
27741  ای دل دو سه شام تا سحرگاه مخسب  در فرقت آفتاب چون ماه مخسپ
FIXED
                                   m1                          m2
27741  ای دل دو سه شام تا سحرگاه مخسب  در فرقت آفتاب چون ماه مخسب


#### creating start, end word

In [79]:
# Start , Ends Cleaning


def clean_and_select(string: str, mode: Literal["start", "end"] = "start"):
    clean_dict = {
        "آ": "ا",
        "ء": "ا",
        "ة": "ت",
    }
    striped = string.strip(" \n\t.؟!،”:()«»")

    match mode:
        case "start":
            res = clean_dict.get(striped[0], striped[0])
        case "end":
            res = clean_dict.get(striped[-1], striped[-1])
        case _:
            raise ValueError(f'mode: Literal["start", "end"] (got {mode})')

    return res

In [80]:
df["start"] = df.m1.apply(partial(clean_and_select, mode="start"))
df["end"] = df.m2.apply(partial(clean_and_select, mode="end"))

df

,index,m1,m2,poet,url,start,end
0,0,تا نشانده‌ام در دل ساق سرو و سیمینت,چیده‌ام به هر دستی میوه‌های سیمین را,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...,ت,ا
1,1,الا ای رو ترش کرده که تا نبود مرا مدخل,نبشته گرد روی خود صلا نعم الادام الخل,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...,ا,ل
2,2,تر کنم از آب چشم روی چونان خشک را,چون جگری بیش نیست سوخته بر خوان دل,سیف فرغانی,https://ganjoor.net/seyf/divan-seyf/ghazalha/s...,ت,ل
3,3,ای دل غم زده از عجز تو معلومم شد,که بر تیغ محبت سپر انداخته‌ای,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...,ا,ی
4,4,از می و مطرب مکن، مدعیا منع من,تا غزلی‌تر بود، قول تو خواهم شنید,سلمان ساوجی,https://ganjoor.net/salman/divanss/ghazalss/sh...,ا,د
...,...,...,...,...,...,...,...
74091,74092,صد نامه فرستادم صد راه نشان دادم,یا راه نمی‌دانی یا نامه نمی‌خوانی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...,ص,ی
74092,74093,آن که دل من چو گوی در خم چوگان اوست,موقف آزادگان بر سر میدان اوست,سعدی,https://ganjoor.net/saadi/divan/ghazals/sh95/,ا,ت
74093,74094,خون در دل و ریشهٔ تنم سوخت چنان,کز دیده بجای اشک خاکستر ریخت,ابوسعید ابوالخیر,https://ganjoor.net/abusaeed/robaee-aa/sh44/,خ,ت
74094,74095,گردون برای خیمه خورشید فلکه‌ات,از کوه و ابر ساخته نازیر و سایه‌بان,حافظ,https://ganjoor.net/hafez/ghaside/sh1/,گ,ن


In [81]:
# list of words that starts the m1
starts = df.start.value_counts()
starts, len(df.start.value_counts())

(start
 ا    10538
 ب     8678
 د     6416
 م     5996
 گ     4674
 چ     4538
 ه     3482
 ت     3314
 ز     2835
 ن     2723
 خ     2571
 س     2335
 ک     1972
 ش     1815
 ع     1616
 ر     1582
 ج     1389
 ی     1152
 و     1126
 پ     1073
 ح      822
 ص      773
 ف      699
 غ      526
 ق      518
 ل      471
 ط      314
 ذ       78
 ظ       34
 ض       21
 ث       11
 ژ        4
 Name: count, dtype: int64,
 32)

In [82]:
# list of words that ends the m2
ends = df.end.value_counts()
ends, len(df.end.value_counts())

(end
 د    15633
 ی    13439
 م    10179
 ت     9956
 ن     5814
 ا     4987
 ر     2517
 ش     2362
 ه     2271
 و     2247
 ل     1014
 ز      986
 ب      856
 س      385
 ق      283
 ک      267
 گ      139
 غ      131
 ف       97
 ح       94
 ع       94
 ج       80
 ظ       48
 خ       40
 ث       39
 چ       33
 ص       31
 ض       26
 ذ       24
 ط       24
 Name: count, dtype: int64,
 30)

In [83]:
end_scale = ends.sum() / ends
end_scale

end
د       4.739717
ی       5.513505
م       7.279301
ت       7.442346
ن      12.744410
ا      14.857830
ر      29.438220
ش      31.370025
ه      32.627037
و      32.975523
ل      73.072978
ز      75.148073
ب      86.560748
س     192.457143
ق     261.823322
ک     277.513109
گ     533.064748
غ     565.618321
ف     763.876289
ح     788.255319
ع     788.255319
ج     926.200000
ظ    1543.666667
خ    1852.400000
ث    1899.897436
چ    2245.333333
ص    2390.193548
ض    2849.846154
ذ    3087.333333
ط    3087.333333
Name: count, dtype: float64

In [84]:
# What word not exist in ends of m2?

set(df.start.unique()) - set(df.end.unique())

{'پ', 'ژ'}

In [85]:
#### variety of poetry

dfg = df.groupby("poet")

df_poet = dfg.agg("count").sort_values("m1")

df_poet

,index,m1,m2,url,start,end
poet,,,,,,
خیام,354,354,354,354,354,354
هاتف اصفهانی,559,559,559,559,559,559
رهی معیری,757,757,757,757,757,757
باباطاهر,760,760,760,760,760,760
عبید زاکانی,826,826,826,826,826,826
سیف فرغانی,1334,1334,1334,1334,1334,1334
ابوسعید ابوالخیر,1438,1438,1438,1438,1438,1438
شهریار,1507,1507,1507,1507,1507,1507
امیرخسرو دهلوی,1671,1671,1671,1671,1671,1671


In [86]:
df_poet.index

Index(['خیام', 'هاتف اصفهانی', 'رهی معیری', 'باباطاهر', 'عبید زاکانی',
       'سیف فرغانی', 'ابوسعید ابوالخیر', 'شهریار', 'امیرخسرو دهلوی', 'صائب',
       'عراقی', 'سلمان ساوجی', 'حافظ', 'اوحدی', 'خواجوی کرمانی',
       'محتشم کاشانی', 'فروغی بسطامی', 'سعدی', 'مولوی'],
      dtype='object', name='poet')

In [87]:
#### starts and ends in poetry of poet

sample_poet = df_poet.index[0]
print(sample_poet)
dfg.get_group(sample_poet).start.unique() , len(dfg.get_group(sample_poet).start.unique())

خیام


(array(['م', 'چ', 'ب', 'ا', 'ک', 'و', 'د', 'گ', 'ت', 'پ', 'ق', 'خ', 'ز',
        'ن', 'ر', 'ع', 'س', 'ج', 'ی', 'ل', 'ه', 'ح', 'ش', 'ف', 'ص', 'ض'],
       dtype=object),
 26)

In [88]:
# and ends
dfg.get_group(sample_poet).end.unique() , len(dfg.get_group(sample_poet).end.unique())

(array(['ن', 'ر', 'ه', 'د', 'م', 'ب', 'ی', 'و', 'ت', 'ل', 'ز', 'س', 'گ',
        'ا', 'خ', 'ش'], dtype=object),
 16)

In [89]:
# create counts of words in starts of each poets

starts_dict = {poet: len(dfg.get_group(poet).start.unique()) for poet in df_poet.index}
ends_dict = {poet: len(dfg.get_group(poet).end.unique()) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 16,
 'هاتف اصفهانی': 17,
 'رهی معیری': 15,
 'باباطاهر': 17,
 'عبید زاکانی': 13,
 'سیف فرغانی': 17,
 'ابوسعید ابوالخیر': 22,
 'شهریار': 16,
 'امیرخسرو دهلوی': 21,
 'صائب': 20,
 'عراقی': 18,
 'سلمان ساوجی': 17,
 'حافظ': 23,
 'اوحدی': 19,
 'خواجوی کرمانی': 23,
 'محتشم کاشانی': 29,
 'فروغی بسطامی': 15,
 'سعدی': 18,
 'مولوی': 25}

In [90]:
# create counts of words in starts of each poets

starts_dict = {poet: len(dfg.get_group(poet).start.unique()) for poet in df_poet.index}
ends_dict = {poet: len(dfg.get_group(poet).end.unique()) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 16,
 'هاتف اصفهانی': 17,
 'رهی معیری': 15,
 'باباطاهر': 17,
 'عبید زاکانی': 13,
 'سیف فرغانی': 17,
 'ابوسعید ابوالخیر': 22,
 'شهریار': 16,
 'امیرخسرو دهلوی': 21,
 'صائب': 20,
 'عراقی': 18,
 'سلمان ساوجی': 17,
 'حافظ': 23,
 'اوحدی': 19,
 'خواجوی کرمانی': 23,
 'محتشم کاشانی': 29,
 'فروغی بسطامی': 15,
 'سعدی': 18,
 'مولوی': 25}

In [91]:
df_count_alphabet_start = pd.DataFrame(
    data=starts_dict.values(),
    index=starts_dict.keys(),
    columns=["count_alphabet_start"],
)
df_count_alphabet_end = pd.DataFrame(
    data=ends_dict.values(),
    index=ends_dict.keys(),
    columns=["count_alphabet_end"],
)

df_count_alphabet = df_count_alphabet_start.merge(
    df_count_alphabet_end,
    left_index=True,
    right_index=True,
)

df_count = df_poet.merge(
    df_count_alphabet,
    left_index=True,
    right_index=True,
)[["start", "count_alphabet_start", "count_alphabet_end"]].rename(columns={"start": "poem_count"})

df_count

,poem_count,count_alphabet_start,count_alphabet_end
خیام,354,26,16
هاتف اصفهانی,559,26,17
رهی معیری,757,27,15
باباطاهر,760,27,17
عبید زاکانی,826,29,13
سیف فرغانی,1334,30,17
ابوسعید ابوالخیر,1438,30,22
شهریار,1507,29,16
امیرخسرو دهلوی,1671,29,21
صائب,2849,29,20


In [92]:
from functools import reduce

In [93]:
# create counts of words in starts of each poets

starts_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).start.unique())) for poet in df_poet.index}
ends_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).end.unique())) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 'ابتخدرزسشلمنهوگی',
 'هاتف اصفهانی': 'ابتجدرزسشقلمنهوگی',
 'رهی معیری': 'ابتحدرزشلمنهوکی',
 'باباطاهر': 'اتجدرزسشفلمنهوچگی',
 'عبید زاکانی': 'ابتدرزشلمنهگی',
 'سیف فرغانی': 'ابتجدرزشقلمنهوچکی',
 'ابوسعید ابوالخیر': 'ابتحدرزسشضطعقلمنهوچکگی',
 'شهریار': 'ابتجدرزسظلمنهوکی',
 'امیرخسرو دهلوی': 'ابتجحخدرزسشطعفقلمنهوی',
 'صائب': 'ابتحدرزسشعغقلمنهوکگی',
 'عراقی': 'ابتدرزسشغقلمنهوکگی',
 'سلمان ساوجی': 'ابتدرزسشعغقلمنهوی',
 'حافظ': 'ابتثجحخدرزسشعغفقلمنهوکی',
 'اوحدی': 'ابتحدرزسشقلمنهوچکگی',
 'خواجوی کرمانی': 'ابتحدرزسشصعغفقلمنهوچکگی',
 'محتشم کاشانی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوکگی',
 'فروغی بسطامی': 'ابتدرزشقلمنهوکی',
 'سعدی': 'ابتدذرزسشغقلمنهوگی',
 'مولوی': 'ابتثجحخدرزسشضعغفقلمنهوکگی'}

In [94]:
def remove_common_letters(dictionary):
    common_letters = set.intersection(*map(set, dictionary.values()))

    new_dictionary = {
        key: "".join(filter(lambda letter: letter not in common_letters, value))
        for key, value in dictionary.items()
    }

    return new_dictionary


remove_common_letters(starts_dict)
# remove_common_letters(ends_dict)

{'خیام': 'ض',
 'هاتف اصفهانی': 'غ',
 'رهی معیری': 'طغ',
 'باباطاهر': 'طغ',
 'عبید زاکانی': 'ذضطغ',
 'سیف فرغانی': 'ذطظغژ',
 'ابوسعید ابوالخیر': 'ذضطظغ',
 'شهریار': 'ثذطغ',
 'امیرخسرو دهلوی': 'ذطغژ',
 'صائب': 'ذضطغ',
 'عراقی': 'ذطظغ',
 'سلمان ساوجی': 'ذطظغ',
 'حافظ': 'ثذضطظغ',
 'اوحدی': 'ثذضطغ',
 'خواجوی کرمانی': 'ثذضطظغ',
 'محتشم کاشانی': 'ثذضطظغ',
 'فروغی بسطامی': 'ثذضطظغژ',
 'سعدی': 'ثذضطظغ',
 'مولوی': 'ذضطظغژ'}

In [95]:
poet_score_ends = {}

for poet, words in ends_dict.items():
    poet_score_end = 0
    for word in words:
        poet_score_end += end_scale[word]
    poet_score_ends[poet] = poet_score_end

df_final = df_count.merge(
    pd.DataFrame(
        poet_score_ends.values(),
        index=poet_score_ends.keys(),
        columns=["poet_score_ends"],
    ),
    right_index=True,
    left_index=True,
).sort_values(by=["poet_score_ends"])

In [96]:
df_final["score"] = (
    df_final["count_alphabet_start"]
    * df_final["count_alphabet_end"]
    * df_final["poet_score_ends"]
    / df_final["poem_count"]
)
df_final.sort_values("score")

,poem_count,count_alphabet_start,count_alphabet_end,poet_score_ends,score
فروغی بسطامی,5662,32,15,953.106144,80.800238
سلمان ساوجی,3343,29,17,2221.923818,327.672283
عراقی,3266,29,18,2244.246356,358.694610
سعدی,7281,31,18,5054.066581,387.332667
مولوی,22664,31,25,12112.976872,414.205660
عبید زاکانی,826,29,13,913.858939,417.100266
اوحدی,4799,30,19,4712.216688,559.692334
صائب,2849,29,20,3820.756994,777.830487
رهی معیری,757,27,15,1479.538142,791.562678
شهریار,1507,29,16,3322.236607,1022.904967
